In [1]:
## Getting the list of files
from pathlib import Path

fish_dir = Path('./fishes/Fish_Dataset/Fish_Dataset')
file_path = list(fish_dir.glob(r"**/*.png"))

file_path[:10]

[WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00001.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00002.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00003.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00004.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00005.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00006.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00007.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00008.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00009.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00010.png')]

In [2]:
## Getting the labels of the images
from os import path
import pandas as pd

labels = list(map(lambda f: path.split(path.split(f)[0])[1], file_path))

labels = pd.Series(labels)
labels.unique()

array(['Black Sea Sprat', 'Black Sea Sprat GT', 'Gilt-Head Bream',
       'Gilt-Head Bream GT', 'Hourse Mackerel', 'Hourse Mackerel GT',
       'Red Mullet', 'Red Mullet GT', 'Red Sea Bream', 'Red Sea Bream GT',
       'Sea Bass', 'Sea Bass GT', 'Shrimp', 'Shrimp GT',
       'Striped Red Mullet', 'Striped Red Mullet GT', 'Trout', 'Trout GT'],
      dtype=object)

In [3]:
## Images series
images = pd.Series(file_path).astype(str)
images.head()

0    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
1    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
2    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
3    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
4    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
dtype: object

In [4]:
## Creating dataframe of fishes with their labels
fishes = pd.concat([images, labels], axis=1)
fishes.columns = ["image", "label"]
fishes = fishes[fishes["label"].apply(lambda x: x[-2:] != "GT")].sample(frac=1).reset_index(drop=True)
fishes.shape

(9000, 2)

In [5]:
## Splitting data
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(fishes, test_size=0.1, random_state=15, shuffle=True)

print(train_data.shape)
print(test_data.shape)

(8100, 2)
(900, 2)


In [6]:
print(train_data.label.value_counts())
print("###")
print(test_data.label.value_counts())

Shrimp                911
Black Sea Sprat       905
Red Sea Bream         904
Red Mullet            902
Sea Bass              898
Hourse Mackerel       898
Trout                 897
Striped Red Mullet    893
Gilt-Head Bream       892
Name: label, dtype: int64
###
Gilt-Head Bream       108
Striped Red Mullet    107
Trout                 103
Hourse Mackerel       102
Sea Bass              102
Red Mullet             98
Red Sea Bream          96
Black Sea Sprat        95
Shrimp                 89
Name: label, dtype: int64


In [7]:
## Image Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
input_shape = (224, 224, 3)

train_gen = ImageDataGenerator(rescale=1. / 255, validation_split=0.15)
test_gen = ImageDataGenerator()

train_images = train_gen.flow_from_dataframe(dataframe=train_data,
                                             x_col="image",
                                             y_col="label",
                                             target_size=img_size,
                                             class_mode="categorical",
                                             batch_size=32,
                                             subset="training",
                                             seed=15,
                                             color_mode="rgb",
                                             shuffle=True)

validation_images = train_gen.flow_from_dataframe(dataframe=train_data,
                                                  x_col="image",
                                                  y_col="label",
                                                  target_size=img_size,
                                                  class_mode="categorical",
                                                  batch_size=32,
                                                  subset="validation",
                                                  seed=15,
                                                  color_mode="rgb",
                                                  shuffle=True)

test_images = test_gen.flow_from_dataframe(dataframe=test_data,
                                           x_col="image",
                                           y_col="label",
                                           target_size=img_size,
                                           class_mode="categorical",
                                           batch_size=32,
                                           color_mode="rgb",
                                           shuffle=False)

Found 6885 validated image filenames belonging to 9 classes.
Found 1215 validated image filenames belonging to 9 classes.
Found 900 validated image filenames belonging to 9 classes.


In [8]:
from keras import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten, Dense, Conv2D

model = Sequential([Conv2D(32, (3, 3), activation='relu', strides=(1, 1), input_shape=input_shape),
                    MaxPooling2D(pool_size=(2, 2)), Flatten(),
                    Dense(128, activation='relu'), Dropout(0.1),
                    Dense(128, activation='relu'), Dropout(0.1), Dense(9, activation="softmax")])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 128)               50466944  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                        

In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger

rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", patience=5, verbose=1, factor=0.2, min_lr=0.0001)
early_stop = EarlyStopping(
    monitor='val_loss', patience=1, restore_best_weights=True)
csv_logger = CSVLogger("model_history_log.csv", append=True)

In [11]:
history = model.fit(train_images, validation_data=validation_images, epochs=50, verbose=1,
                    callbacks=[rate_reduction, early_stop, csv_logger])

Epoch 1/50
216/216 [==============================] - 157s 724ms/step - loss: 2.4958 - accuracy: 0.4107 - val_loss: 0.8634 - val_accuracy: 0.7128 - lr: 0.0010
Epoch 2/50
216/216 [==============================] - 97s 449ms/step - loss: 0.5882 - accuracy: 0.7993 - val_loss: 0.2000 - val_accuracy: 0.9564 - lr: 0.0010
Epoch 3/50
216/216 [==============================] - 97s 448ms/step - loss: 0.2196 - accuracy: 0.9278 - val_loss: 0.1355 - val_accuracy: 0.9539 - lr: 0.0010
Epoch 4/50
216/216 [==============================] - 96s 443ms/step - loss: 0.1258 - accuracy: 0.9566 - val_loss: 0.0401 - val_accuracy: 0.9885 - lr: 0.0010
Epoch 5/50
216/216 [==============================] - 95s 440ms/step - loss: 0.0712 - accuracy: 0.9768 - val_loss: 0.0467 - val_accuracy: 0.9852 - lr: 0.0010
Epoch 6/50
216/216 [==============================] - 99s 460ms/step - loss: 0.0661 - accuracy: 0.9784 - val_loss: 0.0326 - val_accuracy: 0.9901 - lr: 0.0010
Epoch 7/50
216/216 [==============================]

In [12]:
import numpy as np

prediction = np.argmax(model.predict(test_images), axis=1)
predicted_labels = test_images.class_indices
predicted_labels = dict((v, k) for k, v in predicted_labels.items())
predictions = [predicted_labels[k] for k in prediction]

In [13]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_test = test_data.label
print(classification_report(y_test, predictions))

print(accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions, normalize='true')

                    precision    recall  f1-score   support

   Black Sea Sprat       1.00      0.95      0.97        95
   Gilt-Head Bream       1.00      0.94      0.97       108
   Hourse Mackerel       1.00      0.99      1.00       102
        Red Mullet       1.00      1.00      1.00        98
     Red Sea Bream       0.99      1.00      0.99        96
          Sea Bass       0.96      1.00      0.98       102
            Shrimp       0.99      1.00      0.99        89
Striped Red Mullet       1.00      0.95      0.98       107
             Trout       0.90      1.00      0.95       103

          accuracy                           0.98       900
         macro avg       0.98      0.98      0.98       900
      weighted avg       0.98      0.98      0.98       900

0.9811111111111112


array([[0.94736842, 0.        , 0.        , 0.        , 0.        ,
        0.04210526, 0.        , 0.        , 0.01052632],
       [0.        , 0.94444444, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.05555556],
       [0.        , 0.        , 0.99019608, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00980392],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.00934579,
        0.        , 0.00934579, 0.95327103, 0.02803738],


In [14]:
from tensorflowjs import converters

converters.save_keras_model(model, "tf-js-model")

print("Saved successfully")

Saved successfully


In [15]:
print(test_images.filenames[:6])
print(predictions[:6])

['fishes\\Fish_Dataset\\Fish_Dataset\\Striped Red Mullet\\Striped Red Mullet\\00167.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Hourse Mackerel\\Hourse Mackerel\\00009.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Hourse Mackerel\\Hourse Mackerel\\00098.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Shrimp\\Shrimp\\00276.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Shrimp\\Shrimp\\00338.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00193.png']
['Striped Red Mullet', 'Hourse Mackerel', 'Hourse Mackerel', 'Shrimp', 'Shrimp', 'Black Sea Sprat']
